# CH8 객체 참조, 가변성, 재활용

## 8.1 변수는 상자가 아니다
* 변수는 상자 X, 포스트잇
* 본질은 객체, 변수는 객체에 할당해주는 레이블 (별명)

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


In [2]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

In [3]:
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 139676032995408
Gizmo id: 139676032958416


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [4]:
dir()

['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

## 8.2 정체성, 동질성, 별명
* 변수는 객체의 별명 (aliasing)
* 객체의 정체성은 id() 함수를 통해 확인 가능, id() 값은 객체가 소멸할때까지 불변. 비교할때는 비교연산자 is 를 통해 비교.
* 객체의 동질성은 __eq__() 연산자에 정의된 내용으로 확인 가능 (class 를 작성한 개발자가 오버라이딩하여 구현)

In [5]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print(lewis is charles)
print(id(lewis), id(charles))
lewis['balance'] = 950
print(charles)

True
139676032424928 139676032424928
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [6]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
print(alex == charles)
print(alex is not charles)

True
True


### 8.2.2 튜플의 상대적 불변성
* 튜플 자체는 불변형이지만, 참조된 항목(element) 는 변할 수 있음

In [7]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))
print(t1 == t2)

True
139676033381312
(1, 2, [30, 40, 99])
139676033381312
False


## 8.3 기본 복사는 얕은 복사
* 자료 복사하는 가장 쉬운 방법은 내장 생성자를 이용하는 방법

In [80]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l3 = l1[:]
print(l2)
print(l2 == l1)
print(l2 is l1)
print(l3 == l1)
print(l3 is l1)

[3, [55, 44], (7, 8, 9)]
True
False
True
False


* 이경우, 얕은 사본이 되어 가변 항목이 포함된 경우 문제 발생!

In [81]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
##
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)
print(id(l1), id(l2))
print(id(l1[0]), id(l2[0]))
print(id(l1[1]), id(l2[1]))
print(id(l1[2]), id(l2[2]))

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]
139676032578736 139676032577856
94876173525856 94876173525856
139676032577776 139676032577776
139676078076544 139676033342544


* p300 그림 참조!
### 8.3.1 객체의 깊은 복사와 얕은 복사
* copy 모듈의 deepcopy() 함수를 쓰면 위에 문제 해결 가능

In [10]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [13]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))
bus1.drop('Bill')
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print(bus3.passengers)

139676032453520 139676032439248 139676032441168
['Alice', 'Claire', 'David']
139676078314832 139676078314832 139676078315232
['Alice', 'Bill', 'Claire', 'David']


In [84]:
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)

[10, 20, [[...], 30]]


In [85]:
from copy import deepcopy
c = deepcopy(a)
print(c)

[10, 20, [[...], 30]]


In [86]:
print(id(a), id(c))
print(id(a[0]), id(c[0]))
print(id(a[1]), id(c[1]))
print(id(a[2]), id(c[2]))
print(id(a[2][0]), id(c[2][0]))
print(id(a[2][1]), id(c[2][1]))


139676033355120 139676033293280
94876173526080 94876173526080
94876173526400 94876173526400
139676032423568 139676033242128
139676033355120 139676033293280
94876173526720 94876173526720


## 8.4 참조로서의 함수 매개변수
* 파이썬은 공유로 호출하는 매개변수 전달 방식
* 각 함수의 매개변수가 인수로 전달받은 각 참조의 사본을 받음
* 객체를 변경할수는 있지만, 정체성을 바꿀수는 없음

In [17]:
def f(a, b):
    a += b
    return a

In [19]:
x = 1
y = 2
print(f(x, y))
print(x, y)

3
1 2


In [20]:
a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)

[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]


In [21]:
c = [1, 2]
d = [3, 4]
print(f(c, d))
print(c, d)

[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]


In [22]:
t = (10, 20)
u = (30, 40)
print(f(t, u))
print(t, u)

(10, 20, 30, 40)
(10, 20) (30, 40)


### 8.4.1 가변형을 매개변수 기본값으로 사용하기: 좋지 않은 생각
* 매개변수 기본값으로 가변 객체를 사용하면 문제가 발생할수 있음

In [37]:
class HauntedBus:
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [38]:
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charlie')
bus1.drop('Alice')
print(bus1.passengers)

['Alice', 'Bill']
['Bill', 'Charlie']


In [39]:
bus2 = HauntedBus()
bus2.pick('Carrie')
print(bus2.passengers)

bus3 = HauntedBus()
print(bus3.passengers)
bus3.pick('Dave')
print(bus2.passengers)
print(bus2.passengers is bus3.passengers)

['Carrie']
['Carrie']
['Carrie', 'Dave']
True


In [40]:
print(bus1.passengers)

['Bill', 'Charlie']


In [41]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [42]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [43]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### 8.4.2 가변 매개변수에 대한 방어적 프로그래밍
* 전달된 인수가 변경될 것인지 예상할수 있는지 없는지 알아야 함.
* 함수 구현자와 함수 호출자가 예상하는 것을 일치 시켜야 함.

In [44]:
class TwilightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [45]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(basketball_team)

['Sue', 'Maya', 'Diana']


* 전달받은 인자의 사본을 만들어서 할당하면 안전

In [46]:
class TwilightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [47]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(basketball_team)

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']


## 8.5 del 과 가비지 컬렉션
* del은 변수만 제거, 객체를 제거하는 것이 아님.
* 참조를 잃어버린 객체는 가비지 컬렉션에 의해 조용히 삭제됨.

In [49]:
import weakref
s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

# s1 에 대한 약한 참조 ??
ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)


True
True


In [50]:
s2 = 'spam'

Gone with the wind...


In [51]:
print(ender.alive)

False


## 8.6 약한 참조
* 약한 참조는 참조 카운트를 증가시키지 않고 객체를 참조
* 캐시 애플리케이션에 유용하게 사용

In [91]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x7f08dc48f2f0; to 'set' at 0x7f08dc48d050>

In [92]:
wref()

{0, 1}

In [93]:
a_set = {2, 3, 4}
wref()

{0, 1}

In [94]:
print(wref() is None)
print(wref() is None)
## 터미널 환경에서는 잘 됨.

False
False


In [69]:
import weakref
c_set = {0, 1}
wref = weakref.ref(c_set)
wref
wref()
c_set = {2, 3, 4}
wref()
wref() is None
# print(wref() is None)
# print(wref() is None)

True

## 8.6.1 WeakValueDictionary 촌극
* 객체에 대한 약한 참조를 값으로 가지는 가변 매핑을 구현한 클래스
* 참조된 객체가 다른곳에서 가비지 컬렉트되면 해당 키도 WeakValueDictionary에서 제거

In [64]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [71]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys()))
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
[]


### 8.6.2 약한 참조의 한계
* list, dict 객체는 약한 참조 대상이 될수 없지만, 다음과 같이 해결 가능??

In [74]:
class MyList(list):
    pass

In [75]:
a_list = MyList(range(10))
wref_to_a_list = weakref.ref(a_list)

In [95]:
b_list = [1,2,3]
weakref.ref(b_list)

TypeError: cannot create weak reference to 'list' object

## 8.7 파이썬의 특이한 불변형 처리법
* 튜플은 내장 생성자 혹은 [:] 를 사용해도 사본을 생성하지 않음
* str, bytes, frozenset 도 동일하게 동작
* 문자열의 경우, 인터닝 기법에 의해 문자열 리터럴 공유

In [76]:
t1 = (1, 2, 3)
t2 = tuple(t1)
print(t2 is t1)
t3 = t1[:]
print(t3 is t1)

True
True


In [77]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
print(t3 is t1)
s1 = 'ABC'
s2 = 'ABC'
print(s1 is s2)

False
True


In [97]:
fs = frozenset()
print(fs is fs.copy())

True
